In [41]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['localhost'],
    http_auth=('elastic', 'changeme'),
    port=9200
)
print es.indices

In [43]:
es.indices.delete(index='trend', ignore=[400, 404])

{u'error': {u'index': u'trend',
  u'index_uuid': u'_na_',
  u'reason': u'no such index',
  u'resource.id': u'trend',
  u'resource.type': u'index_or_alias',
  u'root_cause': [{u'index': u'trend',
    u'index_uuid': u'_na_',
    u'reason': u'no such index',
    u'resource.id': u'trend',
    u'resource.type': u'index_or_alias',
    u'type': u'index_not_found_exception'}],
  u'type': u'index_not_found_exception'},
 u'status': 404}

In [2]:
inputFile = "/home/foodmap/food-tweets-2017-04-23.json.gz"

In [3]:
import argparse
import os
import sys
import json
import time
os.chdir("/home/foodmap/food101/")
sys.path.append(os.getcwd())
import requests
from elasticsearch import Elasticsearch
from processing.preprocess_tweet import process_tweet
from processing.twitter.Tweet import Tweet


All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251


In [ ]:
print time.time()
tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
numIndex = 0
for tweet in tweetsAsDict:
    
    i += 1
    if i % 10000 == 0:
        print "Processed tweets: ", i
        print "Indexed tweets: ", numIndex

    new_tweet = process_tweet(tweet, forStream=False)
    if new_tweet is None:
        continue
    # new_tweet_id = new_tweet["id"]
#     print tweet
#     print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
#     print new_tweet
    
    # check len of img_categ
    if new_tweet["img_categories"] is not None and len(new_tweet["img_categories"]) > 0:
        new_tweet["img_category"] = new_tweet["img_categories"][0]["label"]
        new_tweet["img_category_score"] = new_tweet["img_categories"][0]["score"]
    else:
        new_tweet["img_category"] = None
        new_tweet["img_category_score"] = 0.0

    # check len of text_categ
    if len(new_tweet["text_categories"]) != 0:
        print new_tweet["text_categories"]
        for cat in new_tweet["text_categories"]:
            new_tweet["text_category"] = cat
            print "Has text categories" 
            print "Id: ", numIndex
            print "--------------------------"

            # split index per month
            es.index(index="trends04", doc_type='tweet', id=numIndex, body=new_tweet)
            numIndex += 1
    else:
        new_tweet["text_category"] = None
        # split index per month
        print "Doesn't have text categories" 
        print "Id: ", numIndex
        print "--------------------------"
        es.index(index="trends04", doc_type='tweet', id=numIndex, body=new_tweet)
        numIndex += 1

In [20]:
from processing.location.get_location_from_tweet import getUserLocation, inferCountryFromCity, getLocationData, \
    getFinalUserLocation
from processing.load_keyword_dicts import loadCategoryDict
from processing.location.locations import Cities, Countries

citiesIndex, citiesInfo = Cities.loadFromFile()
countriesIndex, countriesInfo = Countries.loadFromFile()
ccDict = Countries.countryCodeDict(countriesInfo)

print ccDict.keys()
print ccDict["JM"]


def get_media_url(tweet):
    """
    Checks if the tweets has media file attached
    :param tweet:
    :return:
    """
    if "entities" in tweet:
        if "media" in tweet["entities"]:
            foundMedia = tweet["entities"]["media"]
            return foundMedia[0]["media_url"]
        else:
            return None
    else:
        return None
    
    
def get_image_categories(img_url):
    """
    Gets the food category for a given img url.
    :param img_url:
    :return: list of dicts!!!!
    """
    request_string = 'http://test.tripbuilder.isti.cnr.it:8080/FoodRecognition/services/IRServices/recognizeByURL?imgURL='
    res = requests.get(request_string + img_url)
    try:
        candidates = json.loads(res.text)["guessed"]
    except:
        print "Error! Couldn't parse json ", res.text
        return None
    return candidates


def get_location(tweet):
    """
    Gets the location from the Place/GPS or user location
    :param tweet:
    :return:
    """
    tweet_coords, tweet_place_city, tweet_place_country, tweet_place_country_code, user_loc = getLocationData(tweet)

    if (tweet_place_country is None):
        user_cities, user_countries = getUserLocation(user_loc, citiesIndex, citiesInfo, countriesIndex, countriesInfo)
        inferred_countries = inferCountryFromCity(user_cities, citiesIndex, citiesInfo, ccDict)
        city, country = getFinalUserLocation(user_cities, user_countries, inferred_countries)
    else:
        city = tweet_place_city
        country = tweet_place_country

    return city, country, tweet_coords

All cities with all name:  22309
All cities unique geonameid:   23516
All countries with all names:  24782
All countries unique geonameid:   251
Len of country code dict 251
[u'BD', u'BE', u'BF', u'BG', u'BA', u'BB', u'WF', u'BL', u'BM', u'BN', u'BO', u'JP', u'BI', u'BJ', u'BT', u'JM', u'BV', u'BW', u'WS', u'BQ', u'BR', u'BS', u'BY', u'BZ', u'RU', u'RW', u'RS', u'LT', u'RE', u'LU', u'LR', u'RO', u'LS', u'GW', u'GU', u'GT', u'GS', u'GR', u'GQ', u'GP', u'BH', u'GY', u'GG', u'GF', u'GE', u'GD', u'GB', u'GA', u'GN', u'GM', u'GL', u'GI', u'GH', u'OM', u'TN', u'JO', u'HR', u'HT', u'SV', u'HK', u'HN', u'HM', u'VE', u'PR', u'PS', u'PW', u'PT', u'KR', u'PY', u'IQ', u'PA', u'PF', u'PG', u'PE', u'PK', u'PH', u'PN', u'PL', u'PM', u'ZM', u'EH', u'EE', u'EG', u'ZA', u'EC', u'IT', u'AO', u'SB', u'AW', u'ET', u'SO', u'ZW', u'SA', u'ES', u'ER', u'ME', u'MD', u'MG', u'MF', u'UY', u'MC', u'UZ', u'MM', u'ML', u'MO', u'MN', u'MH', u'MK', u'UM', u'MT', u'MW', u'MV', u'MQ', u'MP', u'MS', u'MR', u'AU', u'UG',

In [ ]:
print time.time()
tweetsAsDict = Tweet.getTweetAsDictionary(inputFile)
i = 0
numImgCat = 0
forStream = False
for tweet in tweetsAsDict:
    
    i += 1
    if i % 10000 == 0:
        print "Processed tweets: ", i
        print "Indexed tweets: ", numIndex
        
    new_tweet = dict()
        
    city, country, tweet_coords = get_location(tweet)
    new_tweet["coords"] = tweet_coords
    if tweet["place"] is not None:
        if "bounding_box" in tweet["place"].keys():
            new_tweet["bounding_box"] = tweet["place"]["bounding_box"]
        else:
            new_tweet["bounding_box"] = None
    else:
        new_tweet["bounding_box"] = None
    new_tweet["city"] = city
    new_tweet["country"] = country

    if forStream:
        if (new_tweet["coords"] is None) and (new_tweet["bounding_box"] is None):
            continue
    else:
        if new_tweet["country"] is None:
            continue


    # IMAGE
    media_url = get_media_url(tweet)
    new_tweet["media_url"] = media_url
    new_tweet["img_flag"] = False
    if media_url is not None:
        img_categories = get_image_categories(media_url)
        new_tweet["img_categories"] = img_categories
        numImgCat += 1
    if numImgCat % 100 == 0:
        print "Processed 100 tweets", time.time()


1498570306.53
Opening file:  /home/foodmap/food-tweets-2017-04-23.json.gz
Error! Couldn't parse json  <!doctype html><html lang="en"><head><title>HTTP Status [500] – [Internal Server Error]</title><style type="text/css">h1 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:22px;} h2 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:16px;} h3 {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;font-size:14px;} body {font-family:Tahoma,Arial,sans-serif;color:black;background-color:white;} b {font-family:Tahoma,Arial,sans-serif;color:white;background-color:#525D76;} p {font-family:Tahoma,Arial,sans-serif;background:white;color:black;font-size:12px;} a {color:black;} a.name {color:black;} .line {height:1px;background-color:#525D76;border:none;}</style></head><body><h1>HTTP Status [500] – [Internal Server Error]</h1><hr class="line" /><p><b>Type</b> Status Report</p><p><b>Message</b> Internal Server Err

In [32]:
es.delete_by_query(index='trends04', doc_type='tweet', body={
    'query': {
        "match_all" : {}
    }
})

{u'batches': 15,
 u'deleted': 14950,
 u'failures': [],
 u'noops': 0,
 u'requests_per_second': -1.0,
 u'retries': {u'bulk': 0, u'search': 0},
 u'throttled_millis': 0,
 u'throttled_until_millis': 0,
 u'timed_out': False,
 u'took': 585,
 u'total': 14950,
 u'version_conflicts': 0}

In [ ]:
es.count(index='trends04', doc_type='tweet')

In [27]:
# Search tweet text for QUERY
es.search(index="trends", body={"query": {"match": {'match_all':'lasagna'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 2}

In [ ]:
# Search tweet for ID
es.get(index='trends', doc_type='tweet', id=855919564705673216)
# http://foodmap.isti.cnr.it:9200/trends/tweet/856023708892360704

In [ ]:
# dealing with the index 
r = requests.get('http://foodmap.isti.cnr.it:9200/',  auth=('elastic', 'changeme'))